In [1]:
%load_ext autoreload
%autoreload 2

In [20]:
import os
import json

import torch
from tqdm import tqdm

from model import get_m1_model
from word_generators import GreedyGenerator
from tokenizers import CharLevelTokenizerv1, KeyboardTokenizerv1
from dataset import NeuroSwipeDatasetv1

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
IN_KAGGLE = False

if IN_KAGGLE:
    DATA_ROOT = "/kaggle/input/yandex-cup-playground"
    MODELS_DIR = ""
else:
    DATA_ROOT = "../data/data_separated_grid"
    MODELS_DIR = "../data/trained_models/m1"

In [36]:
default_model_fname = "best_model__2023_11_04__18_31_37__0.02530_default_switch_2.pt"
extra_model_fname = "best_model__2023_11_05__07_55_13__0.02516_extra_switch_2__with_pad_cutting.pt"

grid_name_to_model = {
    "default": get_m1_model(os.path.join(MODELS_DIR, default_model_fname), device),
    "extra": get_m1_model(os.path.join(MODELS_DIR, extra_model_fname), device)
}

In [37]:
word_char_tokenizer = CharLevelTokenizerv1(os.path.join(DATA_ROOT, "voc.txt"))

In [38]:
grid_name_to_greedy_generator = {
    grid_name: GreedyGenerator(grid_name_to_model[grid_name], word_char_tokenizer, device)
    for grid_name in ("default", "extra")
}

In [42]:
grid_name_to_grid_path = os.path.join(DATA_ROOT, "gridname_to_grid.json")
with open(grid_name_to_grid_path, "r", encoding="utf-8") as f:
    grid_name_to_grid = json.load(f)

val_path = os.path.join(DATA_ROOT, f"valid__in_train_format.jsonl")

kb_tokenizer = KeyboardTokenizerv1()
word_tokenizer = CharLevelTokenizerv1(os.path.join(DATA_ROOT, "voc.txt"))


val_dataset = NeuroSwipeDatasetv1(
    data_path = val_path,
    gridname_to_grid = grid_name_to_grid,
    kb_tokenizer = kb_tokenizer,
    max_traj_len = 299,
    word_tokenizer = word_tokenizer,
    include_time = False,
    include_velocities = True,
    include_accelerations = True,
    has_target=True,
    has_one_grid_only=False,
    include_grid_name=True,
    total = 10_000
)

100%|██████████| 10000/10000 [00:01<00:00, 5443.77it/s]


In [55]:
def create_pred_list_greedy_witout_vocab_check(dataset,
                                               grid_name_to_greedy_generator):
    """
    Creates submission file generating words greedily.

    If prediction is not in the vocabulary 
    """

    preds = []

    for i, data in tqdm(enumerate(dataset), total=len(dataset)):
        try:
            (xyt, kb_tokens, _, traj_pad_mask, word_mask), target, grid_name = data
            pred = grid_name_to_greedy_generator[grid_name](xyt, kb_tokens, traj_pad_mask)
            pred = pred.removeprefix("<sos>") 
            target_len = torch.sum(~word_mask)
            target = word_char_tokenizer.decode(target[:target_len]).removesuffix('<pad>').removesuffix('<eos>')
            print(pred, target)
            preds.append(pred)
        except KeyboardInterrupt:
            print('Досрочно остановлено пользователем')
            break
    return preds

In [56]:
create_pred_list_greedy_witout_vocab_check(val_dataset, grid_name_to_greedy_generator)

  0%|          | 2/10000 [00:00<27:02,  6.16it/s]

на на
все все


  0%|          | 3/10000 [00:00<36:17,  4.59it/s]

этом этом


  0%|          | 4/10000 [00:01<53:08,  3.14it/s]

...

  1%|          | 120/10000 [00:42<1:11:27,  2.30it/s]

надостью надеюсь


  1%|          | 120/10000 [00:42<58:34,  2.81it/s]  

Досрочно остановлено пользователем


['на',
 'все',
 'этом',
 'добрый',
 'девочка',
 'сказала',
 'скинь',
 'гееев',
 'тобой',
 'быстра',
 'есть',
 'да',
 'маж',
 'щас',
 'она',
 'проблема',
 'билайн',
 'уже',
 'раньше',
 'нам',
 'щас',
 'купил',
 'ты',
 'зовут',
 'короче',
 'лучше',
 'приедем',
 'размыто',
 'давай',
 'ты',
 'отдать',
 'привет',
 'не',
 'да',
 'будете',
 'связи',
 'клывань',
 'меня',
 'напиши',
 'знаю',
 'мамой',
 'не',
 'ты',
 'только',
 'они',
 'саминг',
 'спи',
 'соскучилась',
 'целую',
 'что',
 'почему',
 'шакалы',
 'мне',
 'свидимся',
 'ходил',
 'погодим',
 'подъезжать',
 'делишки',
 'ручки',
 'прохладно',
 'хз',
 'купальники',
 'вчера',
 'друг',
 'без',
 'ничего',
 'александровай',
 'сейчас',
 'запихала',
 'че',
 'только',
 'ты',
 'волос',
 'артем',
 'мама',
 'ура',
 'вообще',
 'вот',
 'такто',
 'шорту',
 'бак',
 'че',
 'сегодня',
 'остановились',
 'можно',
 'буду',
 'ты',
 'все',
 'ты',
 'уровенно',
 'мото',
 'последний',
 'ты',
 'ты',
 'вставать',
 'но',
 'другое',
 'ясно',
 'купи',
 'спать',
 'рес